In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import glob              # easy file searching
from PIL import Image    # image loading
import matplotlib.pyplot as plt
import scipy.io
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from torchvision.transforms import ToTensor
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
import h5py
import math
import os
import torchvision as tv # data augmentation/loading utilities
from sklearn.model_selection import train_test_split
import scipy.io
import copy
import time
from tqdm.autonotebook import tqdm
from tempfile import TemporaryDirectory

<ipython-input-2-8230967616de>:22: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
# config
from easydict import EasyDict as edict
__C = edict()
__C.running_length = 10
cfg = __C
# __C.BATCH_SIZE = 64
BATCH_SIZE = 64
NUM_EPOCHS = 20
HISTORY_LENGTH = 10
PREDICTION_LENGTH = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
LEARNING_RATE = 0.3
FRAME_SIZE = 25 # 25 frames from 250 frames per second
FRAME_RATE = int(250/FRAME_SIZE)

cuda


In [ ]:
class Traces_6DOF_Dataset(torch.utils.data.Dataset):
  def __init__(self, path, transform=None, target_transform=None):
    super().__init__()
    # get filenames
    self.files = glob.glob(path)[0]
    self.transform = transform
    self.target_transform = target_transform


    # load grayscale images
    
    self.data = np.array(scipy.io.loadmat(self.files)['MMs1'],dtype='float32')[::FRAME_RATE,:]
    # normalization
    self.data[:,0] = self.data[:,0]/6
    self.data[:,1] = self.data[:,1]/4
    self.data[:,2] = self.data[:,2]/6
    self.data[:,3] = self.data[:,3]/180
    self.data[:,4] = self.data[:,4]/90
    self.data[:,5] = self.data[:,5]/90
    # define your transform
    # self.transform = tv.transforms.ToTensor()

  def __len__(self):
    return len(self.data)-(HISTORY_LENGTH+PREDICTION_LENGTH)*FRAME_SIZE

  def __getitem__(self, i):
    input = self.data[i:i+HISTORY_LENGTH*FRAME_SIZE,:]
    label = self.data[i+HISTORY_LENGTH*FRAME_SIZE:i+(HISTORY_LENGTH+PREDICTION_LENGTH)*FRAME_SIZE,:]
    # transform
    self.transform(input)
    self.target_transform(label)
    return input, label

In [ ]:
train_dataset = Traces_6DOF_Dataset('/content/drive/MyDrive/Traces_6DOF_NJIT/node2mobility.mat', transform=ToTensor(), target_transform=ToTensor())
val_dataset = Traces_6DOF_Dataset('/content/drive/MyDrive/Traces_6DOF_NJIT/node1mobility.mat', transform=ToTensor(), target_transform=ToTensor())
test_dataset = Traces_6DOF_Dataset('/content/drive/MyDrive/Traces_6DOF_NJIT/node5mobility.mat', transform=ToTensor(), target_transform=ToTensor())
# print(dataset[0])
# train_data, test_data = train_test_split_dataset(dataset, test_size=0.25)
print("train dataset size: ", len(train_dataset))

train dataset size:  2625


In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

In [ ]:
print("train data shape[n_batch, seq_len, embedding_size]: ", len(train_dataloader), train_dataset[0][0].shape)

train data shape[n_batch, seq_len, embedding_size]:  41 (250, 6)


In [ ]:
# utilis
# checkpointing
def save_ckpt(fn, network, optimizer):
  torch.save({'net': network.state_dict(), 'opt': optimizer.state_dict()}, fn)

def load_ckpt(fn, network, optimizer):
  # ckpt = torch.load(fn,map_location=torch.device('cpu'))
  ckpt = torch.load(fn)
  network.load_state_dict(ckpt['net'])
  optimizer.load_state_dict(ckpt['opt'])
  return network, optimizer

In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
            batch_first=False
        )

        self.MLP = nn.Sequential(
          nn.Linear(6, 64),
          nn.ReLU(),
          nn.LayerNorm(64),
          nn.Linear(64, 128),
          nn.ReLU(embedding_size),
          nn.LayerNorm(128),
          nn.Linear(128, embedding_size),
        )
        self.dropout = nn.Dropout(dropout)
        self.fc_out = nn.Linear(embedding_size, 6)

    def make_src_mask(self, src):
        src_mask = torch.Tensor(src.cpu().detach().numpy() == np.zeros((1,embedding_size),float))

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_length, N,_ = src.shape
        trg_seq_length, N,_ = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )
        # print("src position shape before pos: ", src_positions.shape)

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )
        embed_src = self.dropout(
            self.MLP(src) + self.src_position_embedding(src_positions)
        )
        # print("src: ", src[0][0])
        # print("position embedding: ", self.src_position_embedding(src_positions)[0][0])
        # print("embed src: ", embed_src[0][0])
        embed_trg = self.dropout(
            self.MLP(trg) + self.trg_position_embedding(trg_positions)
        )

        src_padding_mask = self.make_src_mask(embed_src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )
        # print("trg_mask: ", trg_mask)
        # print("src_mask shape after pos: ", src_padding_mask.shape)
        # print("trg_mask shape after pos: ", trg_mask.shape)
        out = self.transformer(
            embed_src,
            embed_trg,
            # src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask
        )
        # print("output", out)
        # print("output dtype before decode: ", out.dtype)
        # print("output shape before decode: ", out.shape)
        out = self.fc_out(out)
        return out

    def encode(self, src: Tensor):
        src_seq_length, N, _ = src.shape
        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        ) 
        embed_src = self.MLP(src) + self.src_position_embedding(src_positions)
        embed_src = embed_src.to(self.device)
        src_padding_mask = self.make_src_mask(embed_src)
        return self.transformer.encoder(embed_src, src_padding_mask)

    def decode(self, trg: Tensor, memory: Tensor, trg_mask: Tensor):
        trg_seq_length, N,_ = trg.shape
        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )
        embed_trg = self.MLP(trg) + self.trg_position_embedding(trg_positions)
        embed_trg = embed_trg.to(self.device)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )
        return self.transformer.decoder(embed_trg, memory,
                          trg_mask)



In [ ]:
# ntokens = 256  # embedding dimentsion of coordinates
input_size = HISTORY_LENGTH*FRAME_SIZE
output_size = PREDICTION_LENGTH*FRAME_SIZE
embedding_size = 200 # embedding dimension
forward_expansion = 512  # dimension of the feedforward network model in ``nn.TransformerEncoder``
num_encoder_layers = 6
num_decoder_layers = 6  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
num_heads = 8  # number of heads in ``nn.MultiheadAttention``
max_len = HISTORY_LENGTH*FRAME_SIZE
dropout = 0.1  # dropout probability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Transformer(
    embedding_size,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
).to(device=DEVICE)
criterion = nn.MSELoss()
lr = LEARNING_RATE  # learning rate
# optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, factor=0.1, patience=10
# )
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
batch_size = BATCH_SIZE
epochs = NUM_EPOCHS

In [ ]:
def train(model: nn.Module, data_loader, optimizer, scaler, writer, step) -> None:
    progress_bar = tqdm(data_loader)
    model.train() # turn on train mode
    total_loss = 0.
    start_time = time.time()
    num_batches = len(data_loader)
    log_interval = num_batches // 5
    return_loss = 0.
    for batch_idx, (data, targets) in enumerate(progress_bar):
        # print("batch index: ", batch_idx)
        # change src, tar shape from N,S,E to S,N,E to match batch_first=false
        data = data.permute(1, 0, 2)
        data = data.to(device=DEVICE)
        # print("data: ", data.shape)
        targets = targets.permute(1, 0, 2)
        targets = targets.to(device=DEVICE)
        # print("targets: ", targets.shape)
        # with torch.cuda.amp.autocast():
        output = model(data, targets)
        optimizer.zero_grad()
        loss = criterion(output, targets)
        # print(loss)
        # scaler.scale(loss).backward()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        # scaler.step(optimizer)
        optimizer.step()
        # scaler.update()
        writer.add_scalar("Training Loss", loss, global_step=step)
        step += 1

        total_loss += loss.item()
        return_loss += loss.item()
        progress_bar.set_postfix_str(f"training loss={loss.item():.3e}|avg training loss={total_loss/(batch_idx+1):.3e}")
        if batch_idx % log_interval == 0 and batch_idx > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            print(f'| epoch {epoch:3d} | {batch_idx:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f}')
            total_loss = 0
            start_time = time.time()
    return return_loss/(batch_idx+1)

def evaluate(model: nn.Module, dataloader: DataLoader) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    with torch.no_grad():
        for (data, targets) in dataloader:
            data = data.permute(1, 0, 2)
            targets = targets.permute(1, 0, 2)
            data = data.to(device=DEVICE)
            targets = targets.to(device=DEVICE)

            output = model(data, targets)
            # output_flat = output.view(-1, output_size)
            total_loss += criterion(output, targets).item()
            # print("val loss: ", total_loss)
            # data = data.permute(1, 0, 2) #.detach().cpu().numpy()
            # output = output.permute(1, 0, 2) #.detach().cpu().numpy()
            # print("output2: ", output[0][0])
    return total_loss / (len(dataloader) - 1)


In [ ]:
LOAD_MODEL = False
# training
if LOAD_MODEL:
    load_ckpt("/content/drive/MyDrive/checkpoints_new20.pt", model, optimizer)

# with TemporaryDirectory() as tempdir:
#     best_model_params_path = os.path.join(tempdir, "best_model_params.pt")
scaler = torch.cuda.amp.GradScaler()
print(scaler)
training_loss = []
validation_loss = []
best_val_loss = float('inf')
writer = SummaryWriter("run/loss_plot")
step = 0
for epoch in range(1, NUM_EPOCHS+1):
    print(f'Epoch #{epoch}')
    epoch_start_time = time.time()
    train_loss = train(model, train_dataloader, optimizer, scaler, writer, step)
    # print(train_loss)
    training_loss.append(train_loss)
    mean_loss = sum(training_loss)/len(training_loss)
    val_loss = evaluate(model, val_dataloader)
    validation_loss.append(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
        f'valid loss {mean_loss:5.2f} | mean loss {mean_loss:8.2f}')
    print('-' * 89)
    if mean_loss < best_val_loss:
        best_val_loss = mean_loss
        # save model
        save_ckpt("/content/drive/MyDrive/checkpoints_new20.pt", model, optimizer)
    # scheduler.step(mean_loss)
    scheduler.step()
# load_ckpt("/content/drive/MyDrive/checkpoints.pt", model, optimizer)
      


In [ ]:
def predict(model, input):
    model.train()
    trg_input = torch.unsqueeze(input[-1], 0)
    num_tokens = len(input[0])
    for _ in range(0,50):
          # Get source mask
          trg_mask = model.transformer.generate_square_subsequent_mask(PREDICTION_LENGTH*FRAME_SIZE).to(
            DEVICE
          )
          pred = model(input, trg_input)
          # print(pred)
          next_item = pred[0] # num with highest probability
          next_item = torch.unsqueeze(next_item,0)
          trg_input = torch.vstack((trg_input, next_item))
    return trg_input[1:]

In [ ]:
input_arr = []
output_arr = []
targets_arr = []
for (data, targets) in test_dataloader:
    input = torch.unsqueeze(data[0],1).to(DEVICE)
    target = torch.unsqueeze(targets[0],1).to(DEVICE)
    pred = predict(model, input)
    input_arr.append(data)
    output_arr.append(pred)
    targets_arr.append(target)

In [ ]:
print(training_loss)
epochs = range(1, len(training_loss)+1)
plt.figure()
plt.plot(epochs, torch.tensor(training_loss).numpy(), label='Training Loss')
# plt.plot(epochs, torch.tensor(validation_loss).numpy(), label='Validation Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.show()

In [ ]:
yaw1 = torch.squeeze(torch.stack(output_arr))[1,:,3].detach().cpu().numpy()*180
pitch1 = torch.squeeze(torch.stack(output_arr))[1,:,4].detach().cpu().numpy()*90
roll1 = torch.squeeze(torch.stack(output_arr))[1,:,5].detach().cpu().numpy()*90
yaw = torch.squeeze(torch.stack(targets_arr))[1,:,3].detach().cpu().numpy()*180
pitch = torch.squeeze(torch.stack(targets_arr))[1,:, 4].detach().cpu().numpy()*90
roll = torch.squeeze(torch.stack(targets_arr))[1,:, 5].detach().cpu().numpy()*90
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
G = ax.scatter(yaw, pitch, roll, c='g', marker='o')
O = ax.scatter(-yaw1, pitch1, roll1, c='y', marker='o')
plt.legend((G,O),('Ground truth','Output'))
ax.set_xlabel('yaw')
ax.set_ylabel('pitch')
ax.set_zlabel('roll')
plt.show()
# fig.savefig('fig/oo_1.png')